Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [1]:
using Plots, ApproxFun

In [2]:
function Base.expm(A::Matrix{T},f::Fun) where {T<:Number}
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [3]:
T = π;a=0.15
t = Fun(identity,0..T)
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ Operator(diagm(fill(ldirichlet(d),n)));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

# we need eye(n) for the initial condition and zeros(n,n) for the
# right-hand side of the ODE

Φ = A\eye(2n,n);  

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [4]:
ΦT = Φ(T)

4×4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [5]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

-1.1102230246251565e-16 - 0.2683546905319051im
2.233456475320139e-17 + 0.2683546905319052im
0.03747531973228935 + 1.0000000000000002im
-0.03747531973228875 + 1.0im

Floquet Multipliers = 

0.6651802570066608 - 0.7466828146465888im
0.6651802570066607 + 0.7466828146465893im
-1.1249427991479373 - 8.613841577802117e-16im
-0.8889340869219569 + 1.0886302842019495e-16im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [6]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

223

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [9]:
t = Fun(identity,0..10T)
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(xsol)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 592.126,35.087 592.122,35.071 592.113,35.0391 592.099,34.9913 592.082,34.9278 592.06,34.8486 592.033,34.7541 592.002,34.6444 591.967,34.5197 591.927,34.3805 
 591.883,34.2269 591.834,34.0595 591.781,33.8787 591.723,33.6848 591.661,33.4785 591.595,33.2601 591.524,33.0304 591.449,32.7899 591.369,32.5393 591.285,32.2793 
 591.197,32.0106 591.104,31.7339 591.007,31.4502 590.905,31.1601 590.799,30.8648 590.689,30.5649 590.574,30.2616 590.454,29.9558 590.331,29.6486 590.203,29.3411 
 590.07,29.0343 589.933,28.7294 589.792,28.4277 589.646,28.1304 589.496,27.8386 589.342,27.5538 589.183,27.2773 589.02,27.0104 588.852,26.7545 588.68,26.5111 
 588.504,26.2817 588.323,26.0677 588.138,25.8707 587.949,25.6922 587.755,25.5338 587.557,25.3971 587.355,25.2838 587.148,25.1955 586.937,25.1338 586.721,25.1004 
 586.501,25.0971 586.277,25.1255 586.049,25.1874 585.816,25.2845 585.579,25.4185 585.337,25.5912 585.091,25.8043 584.841,26.0595 584.587,26.3586 584.328,26.7033 
 584.065,27.0952 583.798,27.5361 583.526,28.0277 583.25,28.5715 582.97,29.1692 582.685,29.8224 582.397,30.5326 582.103,31.3014 581.806,32.1302 581.505,33.0205 
 581.199,33.9735 580.889,34.9908 580.574,36.0734 580.256,37.2227 579.933,38.4398 579.606,39.7258 579.275,41.0817 578.939,42.5084 578.6,44.0067 578.256,45.5776 
 577.908,47.2217 577.555,48.9396 577.199,50.7318 576.838,52.5988 576.473,54.541 576.104,56.5586 575.731,58.6518 575.354,60.8207 574.972,63.0653 574.586,65.3854 
 574.197,67.7808 573.803,70.2513 573.405,72.7964 573.002,75.4157 572.596,78.1085 572.185,80.8741 571.771,83.7118 571.352,86.6207 570.929,89.5999 570.503,92.6481 
 570.072,95.7644 569.637,98.9474 569.198,102.196 568.754,105.508 568.307,108.883 567.856,112.319 567.401,115.815 566.941,119.367 566.478,122.976 566.011,126.638 
 565.539,130.352 565.064,134.116 564.585,137.928 564.101,141.785 563.614,145.685 563.123,149.627 562.628,153.606 562.128,157.622 561.625,161.671 561.118,165.751 
 560.607,169.858 560.092,173.991 559.574,178.146 559.051,182.32 558.524,186.511 557.994,190.714 557.459,194.927 556.921,199.146 556.379,203.367 555.833,207.588 
 555.283,211.804 554.729,216.011 554.172,220.206 553.61,224.384 553.045,228.542 552.476,232.673 551.904,236.776 551.327,240.843 550.747,244.872 550.163,248.856 
 549.575,252.791 548.984,256.673 548.388,260.494 547.789,264.251 547.187,267.937 546.58,271.548 545.97,275.077 545.356,278.519 544.739,281.868 544.118,285.119 
 543.493,288.265 542.864,291.301 542.232,294.221 541.596,297.019 540.957,299.689 540.314,302.227 539.668,304.626 539.017,306.881 538.364,308.987 537.706,310.939 
 537.046,312.732 536.381,314.362 535.713,315.824 535.042,317.113 534.367,318.227 533.689,319.161 533.007,319.913 532.321,320.479 531.632,320.857 530.94,321.045 
 530.244,321.04 529.545,320.841 528.842,320.447 528.136,319.857 527.427,319.071 526.714,318.087 525.998,316.906 525.278,315.529 524.555,313.955 523.829,312.187 
 523.099,310.224 522.366,308.068 521.63,305.721 520.89,303.184 520.148,300.459 519.401,297.549 518.652,294.455 517.899,291.179 517.144,287.723 516.384,284.091 
 515.622,280.285 514.857,276.307 514.088,272.161 513.316,267.848 512.541,263.372 511.763,258.737 510.982,253.946 510.197,249.002 509.409,243.909 508.619,238.672 
 507.825,233.296 507.028,227.785 506.228,222.145 505.425,216.382 504.619,210.503 503.81,204.516 502.998,198.428 502.183,192.25 501.365,185.99 500.544,179.66 
 499.72,173.271 498.893,166.837 498.064,160.371 497.231,153.888 496.395,147.405 495.557,140.939 494.715,134.508 493.871,128.131 493.024,121.828 492.174,115.622 
 491.321,109.532 490.465,103.583 489.607,97.7963 488.746,92.1967 487.882,86.8074 487.015,81.6523 486.146,76.7551 485.273,72.1391 484.399,67.8272 483.521,63.84